# Macro and Markets Dashboard
## Executive Version -- United States

Brian W. Dew (brianwdew@gmail.com), as of January 15, 2017

---

#### Objective:
Using python, obtain economic data from the web and save it as csv and txt files to be read by a LaTex file containing chart formatting. 

In [ ]:
help('modules')


Please wait a moment while I gather a list of all available modules...



C:\Anaconda2\lib\site-packages\IPython\kernel\__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated. You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)
C:\Anaconda2\lib\site-packages\blaze\server\server.py:17: ExtDeprecationWarning: Importing flask.ext.cors is deprecated, use flask_cors instead.
  from flask.ext.cors import cross_origin
C:\Anaconda2\lib\site-packages\matplotlib\cbook.py:136: MatplotlibDeprecationWarning: The matplotlib.delaunay module was deprecated in version 1.4. Use matplotlib.tri.Triangulation instead.
  warnings.warn(message, mplDeprecation, stacklevel=1)
C:\Anaconda2\lib\site-packages\nltk\twitter\__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "
C:\Anaconda2\lib\pkgutil.py:110: FutureWarning: The pandas

In [3]:
# Import libraries
import pandas as pd
import pandas_datareader.data as web
import datetime
import os
import shutil
import quandl
import config   ## File with API key
quandl.ApiConfig.api_key = config.key

os.chdir('C:\Working\Python\Macro_Dash\data')

def write_txt(filename, filetext):
# Write label to txt file
    with open(filename, 'w') as text_file:
        text_file.write(filetext)

ImportError: No module named quandl

### Real GDP 

Currently retrieved from FRED. More ideal would be to use the BEA API.

In [ ]:
# Inputs to pandas datareader:
start = datetime.datetime(2005,1,1)
series = ['GCEC1', 'GPDIC1', 'NETEXC', 'PCECC96', 'GDPC1']

# Retrieve data as pandas dataframe named df
df = web.DataReader(series, 'fred', start)

# Record strings for later use in chart label
date = '{} Q{}'.format(df.index[-1].year, df.index[-1].quarter)
Y = (df['GDPC1'][-1] / 1000).round(1)  # GDP in levels

for s in series:  # Convert to share of total change
    df[s] = ((1 + (df[s] - df[s].shift(1))
              /df['GDPC1'].shift(1))**4 - 1) * 100
    
df[4:].round(1).to_csv('gdp_comp.csv') # Save to csv

# Declare variables for chart label
ch = df['GDPC1'][-1].round(1)  # GDP in growth
label = '{}: Real GDP: {}T; Growth: {}\%'.format(date, Y, ch)

write_txt('gdp_comp.txt', label) # Write chart label to txt

### Unemployment Rates

Collect U3 data from FRED

In [ ]:
start = datetime.datetime(2006,1,1) 
series = ['UNRATE', 'LNS14027659', 'LNS14027662']

# Retrieve data as pandas dataframe named df
df = web.DataReader(series, 'fred', start)
df.to_csv('unemp.csv')  # save to csv

last = df['UNRATE'][-1] # Current U3 rate
date = df.index[-1].strftime('%b %Y')
label = '{}: {}\%'.format(date, last)
write_txt('unemp.txt', label) # label to txt file

### Consumer Price Index (CPI) time series

In [ ]:
# Inputs to the pandas datareader:
start = datetime.datetime(2005,1,1) 
series = ['CPIAUCSL', 'CPILFESL', 'T5YIFR']             
df = web.DataReader(series, 'fred', start)

# Inflation breakeven data as extra chart label
exp_inf = round(df['T5YIFR'].dropna()[-1],1)
exp_dt = df['T5YIFR'].dropna().index[-1].strftime('%b %d, %Y')
label = '{}: {}\%'.format(exp_dt, exp_inf)
write_txt('breakeven.txt', label) # label to txt file

df = df[series[:2]].dropna().pct_change(12) * 100
df[12:].round(1).to_csv('cpi.csv') # Save to csv

# CPI time series chart main label to txt
label = '{}: All-items CPI: {}\%; Core CPI: {}\%'.format(
    df.index[-1].strftime('%b %Y'),
    df['CPIAUCSL'].iloc[-1].round(1), 
    df['CPILFESL'].iloc[-1].round(1))
write_txt('cpi.txt', label) # label to txt file

### CPI recent changes by component

In [ ]:
# Updated inputs to pandas datareader:
start = datetime.datetime(2006,1,1)
series = {'CPIOGSSL': 'Other goods \& serv',
          'CPIEDUSL': 'Education',
          'CPIRECSL': 'Recreation', 
          'CPIFABSL': 'Food \& Beverage', 
          'CPITRNSL': 'Transportation',
          'CPIHOSSL': 'Housing', 
          'CPIENGSL': 'Energy', 
          'CPIMEDSL': 'Healthcare', 
          'CPIAPPSL': 'Apparel'}

# Retrieve data as pandas dataframe named df
df = web.DataReader(series.keys(), 'fred', start)
df.columns = series.values()

# Obtain the annual percent change (inflation rate)
df = df.pct_change(12) * 100
d = {s: df[s][-120:].mean() for s in df.keys()} # 10-yr avg

# Write legend months to txt file
write_txt('cpi_mo1.txt', df.index[-2].strftime('%b %Y'))
write_txt('cpi_mo2.txt', df.index[-1].strftime('%b %Y'))

# Keep only latest two months, transpose, and round
df = df.tail(2).transpose().round(1)
df.columns = ['one', 'two']
df.loc[:]['ten'] = [round(d[x],1) for x in df.index]
df.index.name = 'Item'
df = df.sort_values(by='two', axis=0, ascending=False)
df.to_csv('cpi_comp.csv')  # save to csv

### EPS by sector from S&P

In [ ]:
# Get xlsx data from Standard and Poor's website
spfile = 'https://us.spindices.com/documents/additional-material/sp-500-eps-est.xlsx'
    
df = pd.read_excel(spfile, sheetname='SECTOR EPS', skiprows=61)
df = df.ix[1:11,'INDEX NAME':'2017 Q4']
df = df.set_index('INDEX NAME').dropna(axis=1).ix[:,-21:]

dfs = pd.DataFrame()  # Create new df for chart values
dfs['mark'] = df.iloc[:,-1]
dfs['avg'] = df.ix[:,-4:].mean(axis=1)
dfs['max'] = df.max(axis=1)
dfs['min'] = df.min(axis=1)
dfs['neg'] = dfs.loc[dfs['min'] < 0]['min']
dfs['min'] = dfs.loc[dfs['min'] >= 0]['min']
dfs['max'] = dfs['max'].subtract(dfs['min'], fill_value=0)
dfs.index.names = ['A']
dfs = dfs.reset_index()

dfs.replace({'S&P 500 ': ''}, regex=True, inplace=True)
dfs.replace({'munication': ''}, regex=True, inplace=True)
dfs.replace({'state.*$': 'state'}, regex=True, inplace=True)
dfs = dfs.set_index('A').sort_values('mark', ascending=False)

dfs.fillna('.').to_csv('eps.csv')  # csv file created

write_txt('eps.txt', df.columns.values[-1]) # Latest quarter

### Table of other indicators

In [ ]:
# Series from the FRED pandas DataReader method
fredseries = ['M2OWN', 'MORTGAGE30US', 'INDPRO', 'TWEXBMTH']
start = datetime.datetime(2015,12,1)
freddf = web.DataReader(fredseries, 'fred', start)

# Volatility index (VIX) from CBOE
vixurl = 'http://www.cboe.com/publish/scheduledtask/mktdata/datahouse/vixcurrent.csv'
vixcol = ['Date', 'Open', 'High', 'Low', 'VIXCLS']
vix = pd.read_csv(vixurl, skiprows=3000, names=vixcol, 
                  parse_dates=['Date']).set_index('Date')['VIXCLS']

# Consumer confidence index data from the University of Michigan website
cc_url = 'http://www.sca.isr.umich.edu/files/tbmics.csv'
cc = pd.read_csv(cc_url, parse_dates={'DATE': ['Month', 'YYYY']})[-13:]
cc = cc.set_index('DATE').rename(columns={'ICS_ALL':'cc'})['cc']

# Yahoo Finance data on S&P index
sp = pd.read_csv('C:\Working\USA\datafiles\stocks.csv', parse_dates = ['Date'])
sp = sp.set_index('Date')['sp'].rename('Close').iloc[-350:]

# Quandl data on oil and treasuries
series = ['CHRIS/CME_CL1', 'USTREASURY/YIELD']
subseries = ['USTREASURY/YIELD - 3 MO', 'USTREASURY/YIELD - 2 YR', 
             'USTREASURY/YIELD - 10 YR', 'CHRIS/CME_CL1 - Last']
quandlsrs = quandl.get(series, start_date='2015-12-01')[subseries]
quandlsrs.columns = ['3 MO', '2 YR', '10 YR', 'Last']

df = pd.concat([quandlsrs, sp, vix, cc, freddf], axis=1)

In [ ]:
# Dictionary of table rows and guidance on their order, contents, and format
d = {'3 MO':{'n':4,'name':'3-month treasury bill yield','m_ch':20,'y_ch':252, 't':'diff'},
     '2 YR':{'n':5,'name':'2-year treasury bond yield','m_ch':20,'y_ch':252, 't':'diff'},
     'INDPRO':{'n':10,'name':'Industrial production index','m_ch':1,'y_ch':12, 't':'pct'},
     '10 YR':{'n':6,'name':'10-year treasury bond yield','m_ch':20,'y_ch':252, 't':'diff'},
     'Last':{'n':9,'name':'Crude oil, US\$/barrel','m_ch':20,'y_ch':252, 't':'pct'},
     'M2OWN':{'n':3,'name':'Bank deposit interest rate','m_ch':1,'y_ch':12, 't':'diff'},
     'cc':{'n':11,'name':'Consumer confidence index','m_ch':1,'y_ch':12, 't':'pct'},
     'MORTGAGE30US':{'n':7,'name':'30-year mortgage rate','m_ch':4,'y_ch':52, 't':'diff'},             
     'Close':{'n':1,'name':'S\&P 500 index','m_ch':20,'y_ch':252, 't':'pct'},
     'VIXCLS':{'n':2,'name':'CBOE volatility index (VIX)','m_ch':20,'y_ch':252, 't':'pct'},  
     'TWEXBMTH':{'n':8,'name':'US Dollar, broad index','m_ch':1,'y_ch':12, 't':'pct'},
    }
# LaTeX arrows
upar = '\quad \color{green!80!blue}$\\blacktriangle$\\normalcolor'
dnar = '\quad \color{red!80!orange}$\\blacktriangledown$\\normalcolor'

# This section adds a dictionary entry for the monthly and annual percent change columns
# for each row in the table. Interest rate series get difference rather than pct_change.
for k, v in d.iteritems():
    if d[k]['t'] == 'diff':
        d[k]['val'] = '{:.2f}\%'.format(round(df[k].dropna()[-1],2))
    elif k in ['INDPRO','cc']:
        d[k]['val'] = '{:.1f}'.format(round(df[k].dropna()[-1],1))
    elif k in 'Last':
        d[k]['val'] = '\${:.2f}'.format(round(df[k].dropna()[-1],2))
    else:
        d[k]['val'] = '{:.2f}'.format(round(df[k].dropna()[-1],2))
    if d[k]['m_ch'] == 1:
        d[k]['date'] = df[k].dropna().index[-1].strftime('%b %Y')
    else:
        d[k]['date'] = df[k].dropna().index[-1].strftime('%Y-%m-%d')
    for s in ['y', 'm']:  # Loop for yearly and monthly changes
        # Define perecent change and difference:
        pct_ch = df[k].dropna().pct_change(periods=d[k]['{}_ch'.format(s)])[-1]
        diff_ch = df[k].dropna()[-1] - df[k].dropna()[-d[k]['{}_ch'.format(s)]-1]
        if d[k]['t'] == 'pct':   
            d[k]['{}_ch_v'.format(s)] = pct_ch*100
            d[k]['{}_ch_s'.format(s)] = '{:.1f}\%'.format(round(pct_ch*100,1))
        else: 
            d[k]['{}_ch_v'.format(s)] = diff_ch
            d[k]['{}_ch_s'.format(s)] = '{:.2f}$\; $'.format(round(diff_ch,2))
        if d[k]['{}_ch_v'.format(s)] > 0.005:
            d[k]['{}_ar'.format(s)] = upar # Green up arrow if positive
        elif d[k]['{}_ch_v'.format(s)] < -0.005:
            d[k]['{}_ar'.format(s)] = dnar # Red down arrow if negative
        else:
            d[k]['{}_ar'.format(s)] = '' # For cases with no change

In [ ]:
order = {d[k]['n']: k for k in d.keys()}    
# Write label to txt file

with open('table1.txt', 'w') as text_file:
    for n in range(1,12):
        sd = d[order[n]]
        text_file.write( ' \ {} $\quad$ & {} & {} & {} & {} & {} $\; $& {} \ \\\ '.format(
            sd['name'], sd['val'], sd['m_ar'], sd['m_ch_s'], 
            sd['y_ar'], sd['y_ch_s'], sd['date'])
        ) 

In [ ]:
d